In [17]:
import pandas as pd
import altair as alt
import warnings
warnings.filterwarnings("ignore")
alt.renderers.set_embed_options(theme='light')
pd.set_option("display.max_colwidth", 400)
pd.set_option("display.max_rows", 400)


In [18]:
def claim(claim_hash):
    df = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"])
    df.columns = [c.lower() for c in df.columns]
    return df

In [19]:
class yLunaPriceDP:
    
    def __init__(self, claim):
        self.yluna_price = 'e49168df-26f3-4972-b8c2-309e34e41072'
        self.claim = claim
        
    def load(self):
        yluna_price_df = self.claim(self.yluna_price)
        yluna_price_df.columns = [c.lower() for c in yluna_price_df.columns]
        self.yluna_price_df = yluna_price_df

In [20]:
dp = yLunaPriceDP(claim)
dp.load()

In [21]:
yluna = dp.yluna_price_df[dp.yluna_price_df.offer_asset=='yLuna']
prism = dp.yluna_price_df[dp.yluna_price_df.offer_asset=='PRISM']

In [36]:
df = yluna.merge(prism, on='day', suffixes=['_yluna','_prism'])
df['yluna_price'] = (1/df.belief_price_prism) / df.belief_price_yluna
df.sort_values(by='day')

,ask_asset_yluna,belief_price_yluna,day,offer_asset_yluna,ask_asset_prism,belief_price_prism,offer_asset_prism,yluna_price
32,PRISM,0.014991,2022-02-14 00:00:00.000,yLuna,UST,2.641114,PRISM,25.257616
19,PRISM,0.011916,2022-02-15 00:00:00.000,yLuna,UST,2.270224,PRISM,36.966261
6,PRISM,0.010818,2022-02-16 00:00:00.000,yLuna,UST,2.390299,PRISM,38.672993
10,PRISM,0.010129,2022-02-17 00:00:00.000,yLuna,UST,2.692882,PRISM,36.661442
17,PRISM,0.009011,2022-02-18 00:00:00.000,yLuna,UST,3.053132,PRISM,36.349276
2,PRISM,0.009186,2022-02-19 00:00:00.000,yLuna,UST,2.942624,PRISM,36.995210
0,PRISM,0.009403,2022-02-20 00:00:00.000,yLuna,UST,3.001322,PRISM,35.435485
28,PRISM,0.008662,2022-02-21 00:00:00.000,yLuna,UST,3.038520,PRISM,37.993155
15,PRISM,0.008216,2022-02-22 00:00:00.000,yLuna,UST,3.178994,PRISM,38.287377
21,PRISM,0.007824,2022-02-23 00:00:00.000,yLuna,UST,2.915908,PRISM,43.832131


## Query

In [99]:
import requests
luna_apr = float(
    requests.get('https://api.terra.dev/chart/staking-return/annualized').json(
    )[-1]['value']) * 100
et_query = requests.get(
    'https://api.extraterrestrial.money/v1/api/prices').json()
yluna_price = float(et_query['prices']['yLUNA']['price'])
luna_price = float(et_query['prices']['LUNA']['price'])
yluna_apr = luna_apr * luna_price / yluna_price

In [100]:
yluna_apr

9.56155126835892

In [101]:
luna_apr

7.134311955168918

In [102]:
luna_price

85.8633122257453

In [103]:
df = pd.DataFrame(requests.get('https://api.terra.dev/chart/staking-return/annualized').json())
df['date'] = pd.to_datetime(df['datetime'], unit='ms')
df = df[df['date'] > '2021-07-05 15:00:00']

In [104]:
df.value = df.value.apply(float).apply(lambda x: x*100)

In [105]:
mars_price_chart = alt.Chart(df).mark_line(point = True).encode(
    x=alt.X('date:T'),
    y=alt.X('value:Q'),
    tooltip=[alt.Tooltip('date:T', format='%Y-%m-%d %H:%M'),'value:Q']
).properties(width=800).properties(height=300).configure_view(strokeOpacity=0).interactive()
mars_price_chart

alt.Chart(...)